In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm

from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, r2_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from utils.data_processing import *
from utils.graph_processing import *
from models.logistic_regression import LogisticRegression

We are first loading all possible games: games played before 2011, that were winnable, where there was no timeout. We remove timeouted games as we do not know the reason why the player gave up and it may not be linked with the Wikispeedia game.

In [2]:
games = load_preprocessed_games()
games.head()

Loaded 51318 finished paths in df of shape (51318, 7)
Loaded 24875 unfinished paths in df of shape (24875, 8)
After filtering all paths after 2011-02-07 05:02:15
we kept 23245 paths out of 51318 finished paths
There are 24875 unfinished paths
Loaded 4604 articles in df of shape (4604, 1)
Pruning invalid games. Initially we have 48120 games
Pruned invalid games. Now we have 48092 valid games
After removing timeouted games, there are 38775 games left


,difficulty_rating,duration,finished,hashIP,num_backward,path,path_length,source,target,timestamp,type_end
0,NaN,166,True,6a3701d319fc3754,0,"[14th_century, 15th_century, 16th_century, Pac...",9,14th_century,African_slave_trade,2011-02-15 03:26:49,NaN
1,3.0,88,True,3824310e536af032,0,"[14th_century, Europe, Africa, Atlantic_slave_...",5,14th_century,African_slave_trade,2012-08-12 06:36:52,NaN
2,NaN,138,True,415612e93584d30e,0,"[14th_century, Niger, Nigeria, British_Empire,...",8,14th_century,African_slave_trade,2012-10-03 21:10:40,NaN
3,3.0,175,True,015245d773376aab,0,"[14th_century, Italy, Roman_Catholic_Church, H...",7,14th_century,John_F._Kennedy,2013-04-23 15:27:08,NaN
4,NaN,110,True,5295bca242be81fe,0,"[14th_century, Europe, North_America, United_S...",6,14th_century,John_F._Kennedy,2013-07-03 22:26:54,NaN


Let's see how long games are.

In [3]:
games[(games['path_length'] == 1)].shape

(3204, 11)

In [4]:
print(games[(games['path_length'] == 1) & (games['finished'] == True)].shape)
print(games[(games['path_length'] == 1) & (games['finished'] == True)])

(9, 11)
       difficulty_rating  duration  finished            hashIP  num_backward  \
3908                 NaN         0      True  43f864c75b2571b5             0   
5405                 NaN         0      True  43f864c75b2571b5             0   
11137                NaN         0      True  43f864c75b2571b5             0   
12391                NaN         0      True  43f864c75b2571b5             0   
14685                NaN         0      True  43f864c75b2571b5             0   
15737                NaN         0      True  43f864c75b2571b5             0   
15911                NaN         0      True  43f864c75b2571b5             0   
17716                NaN         0      True  43f864c75b2571b5             0   
22649                NaN         0      True  43f864c75b2571b5             0   

                         path  path_length                source  \
3908                [Lesotho]            1               Lesotho   
5405                   [Coal]            1             

So there are 9 games where the source and target article are the same, so the player automatically won. These games are not useful when analysing games so we will remove them. We still keep the lost games of path length 1 since the nature of the first article may impact if the player gives up.

In [5]:
condition = (games['path_length'] == 1) & (games['finished'] == True)
games = games[~condition]
games.shape

(38766, 11)

In [6]:
games[(games['path_length'] == 2)].shape

(2233, 11)

In [7]:
games[(games['path_length'] == 3)].shape

(3354, 11)

In [8]:
games[(games['path_length'] > 4)].shape

(24641, 11)

Most of the games have a path length bigger than 4 (at least 3 clicks before the final click), so we will first try to analyse these.

In [9]:
games_longer_than_4 = games.copy()
games_longer_than_4 = games_longer_than_4[(games_longer_than_4['path_length'] > 4)]

The features we will take into account will be the inherent difficulty of the game and informations from the first 3 clicks. The information from the first 3 clicks will be: the shortest path length from the article to the goal, the fame of the article, if the click was a backward click or not, the pagerank of the article, the duration (approximated), the probabilities of the link used being known to people, the pagerank of the target. The target variable is whether the games was won or lost. We do not use information from the first article as it is contained in the inherent difficulty.

In [10]:
games_longer_than_4.head()

,difficulty_rating,duration,finished,hashIP,num_backward,path,path_length,source,target,timestamp,type_end
0,NaN,166,True,6a3701d319fc3754,0,"[14th_century, 15th_century, 16th_century, Pac...",9,14th_century,African_slave_trade,2011-02-15 03:26:49,NaN
1,3.0,88,True,3824310e536af032,0,"[14th_century, Europe, Africa, Atlantic_slave_...",5,14th_century,African_slave_trade,2012-08-12 06:36:52,NaN
2,NaN,138,True,415612e93584d30e,0,"[14th_century, Niger, Nigeria, British_Empire,...",8,14th_century,African_slave_trade,2012-10-03 21:10:40,NaN
3,3.0,175,True,015245d773376aab,0,"[14th_century, Italy, Roman_Catholic_Church, H...",7,14th_century,John_F._Kennedy,2013-04-23 15:27:08,NaN
4,NaN,110,True,5295bca242be81fe,0,"[14th_century, Europe, North_America, United_S...",6,14th_century,John_F._Kennedy,2013-07-03 22:26:54,NaN


In [11]:
games_longer_than_4['duration'] = (3/(games_longer_than_4['path_length']-1))* games_longer_than_4['duration']

In [12]:
games_longer_than_4.drop(columns = ["difficulty_rating", 'hashIP', 'num_backward', 'path_length', 'type_end', 'timestamp'], inplace = True)

In [13]:
games_longer_than_4.head()

,duration,finished,path,source,target
0,62.250000,True,"[14th_century, 15th_century, 16th_century, Pac...",14th_century,African_slave_trade
1,66.000000,True,"[14th_century, Europe, Africa, Atlantic_slave_...",14th_century,African_slave_trade
2,59.142857,True,"[14th_century, Niger, Nigeria, British_Empire,...",14th_century,African_slave_trade
3,87.500000,True,"[14th_century, Italy, Roman_Catholic_Church, H...",14th_century,John_F._Kennedy
4,66.000000,True,"[14th_century, Europe, North_America, United_S...",14th_century,John_F._Kennedy


In [14]:
games_longer_than_4['first_click'] = games_longer_than_4['path'].apply(lambda x: x[1])
games_longer_than_4['second_click'] = games_longer_than_4['path'].apply(lambda x: x[2])
games_longer_than_4['third_click'] = games_longer_than_4['path'].apply(lambda x: x[3])
games_longer_than_4['num_back'] = games_longer_than_4.apply(lambda a: int(a['first_click'] == '<') + int(a['second_click'] == '<') + int(a['third_click'] == '<'), axis = 1)


#games_longer_than_4.drop(columns=['path'], inplace=True)

games_longer_than_4.head()


,duration,finished,path,source,target,first_click,second_click,third_click,num_back
0,62.250000,True,"[14th_century, 15th_century, 16th_century, Pac...",14th_century,African_slave_trade,15th_century,16th_century,Pacific_Ocean,0
1,66.000000,True,"[14th_century, Europe, Africa, Atlantic_slave_...",14th_century,African_slave_trade,Europe,Africa,Atlantic_slave_trade,0
2,59.142857,True,"[14th_century, Niger, Nigeria, British_Empire,...",14th_century,African_slave_trade,Niger,Nigeria,British_Empire,0
3,87.500000,True,"[14th_century, Italy, Roman_Catholic_Church, H...",14th_century,John_F._Kennedy,Italy,Roman_Catholic_Church,HIV,0
4,66.000000,True,"[14th_century, Europe, North_America, United_S...",14th_century,John_F._Kennedy,Europe,North_America,United_States,0


In [15]:
print((games_longer_than_4['first_click'] == '<').sum())
print((games_longer_than_4['second_click'] == '<').sum())
print((games_longer_than_4['third_click'] == '<').sum())
games_longer_than_4[games_longer_than_4['num_back'] > 0].shape



0
1235
1303


(2538, 9)

In [16]:
games_longer_than_4['second_is_<'] = games_longer_than_4['second_click'] == '<'
games_longer_than_4['third_is_<'] = games_longer_than_4['third_click'] == '<'

In [17]:
games_longer_than_4['second_click'] = games_longer_than_4.apply(
    lambda row: row['source'] if row['second_click'] == '<' else row['second_click'], axis=1
)

games_longer_than_4['third_click'] = games_longer_than_4.apply(
    lambda row: row['first_click'] if row['third_click'] == '<' else row['third_click'], axis=1
)
print((games_longer_than_4['first_click'] == '<').sum())
print((games_longer_than_4['second_click'] == '<').sum())
print((games_longer_than_4['third_click'] == '<').sum())

0
0
0


In [18]:
node_stats_df = load_or_compute_node_stats()
games_longer_than_4 = merge_with_node_data(games_longer_than_4, node_stats_df, columns = ['source', 'target', 'first_click', 'second_click', 'third_click'], data = ['pagerank'])
games_longer_than_4.head()

Loaded 4604 node stats


,duration,finished,path,source,target,first_click,second_click,third_click,num_back,second_is_<,third_is_<,pagerank_source,pagerank_target,pagerank_first_click,pagerank_second_click,pagerank_third_click
0,62.250000,True,"[14th_century, 15th_century, 16th_century, Pac...",14th_century,African_slave_trade,15th_century,16th_century,Pacific_Ocean,0,False,False,0.000642,0.000030,0.001024,0.001223,0.001408
1,66.000000,True,"[14th_century, Europe, Africa, Atlantic_slave_...",14th_century,African_slave_trade,Europe,Africa,Atlantic_slave_trade,0,False,False,0.000642,0.000030,0.006698,0.003321,0.000145
2,59.142857,True,"[14th_century, Niger, Nigeria, British_Empire,...",14th_century,African_slave_trade,Niger,Nigeria,British_Empire,0,False,False,0.000642,0.000030,0.000408,0.000630,0.001197
3,87.500000,True,"[14th_century, Italy, Roman_Catholic_Church, H...",14th_century,John_F._Kennedy,Italy,Roman_Catholic_Church,HIV,0,False,False,0.000642,0.000315,0.003975,0.002173,0.000357
4,66.000000,True,"[14th_century, Europe, North_America, United_S...",14th_century,John_F._Kennedy,Europe,North_America,United_States,0,False,False,0.000642,0.000315,0.006698,0.002751,0.009774


In [19]:
fame_df = load_fame()
games_longer_than_4 = merge_with_fame_data(games_longer_than_4, fame_df, columns = ['source', 'first_click', 'second_click', 'third_click'])

In [20]:
s = fame_df.isna()
s[s['fame_score'] == True]

,fame_score
article_name,


In [21]:
games_longer_than_4.head()

,duration,finished,path,source,target,first_click,second_click,third_click,num_back,second_is_<,third_is_<,pagerank_source,pagerank_target,pagerank_first_click,pagerank_second_click,pagerank_third_click,fame_score_source,fame_score_first_click,fame_score_second_click,fame_score_third_click
0,62.250000,True,"[14th_century, 15th_century, 16th_century, Pac...",14th_century,African_slave_trade,15th_century,16th_century,Pacific_Ocean,0,False,False,0.000642,0.000030,0.001024,0.001223,0.001408,8.0,8.0,8.0,9.0
1,66.000000,True,"[14th_century, Europe, Africa, Atlantic_slave_...",14th_century,African_slave_trade,Europe,Africa,Atlantic_slave_trade,0,False,False,0.000642,0.000030,0.006698,0.003321,0.000145,8.0,8.0,9.0,6.0
2,59.142857,True,"[14th_century, Niger, Nigeria, British_Empire,...",14th_century,African_slave_trade,Niger,Nigeria,British_Empire,0,False,False,0.000642,0.000030,0.000408,0.000630,0.001197,8.0,6.0,8.0,8.0
3,87.500000,True,"[14th_century, Italy, Roman_Catholic_Church, H...",14th_century,John_F._Kennedy,Italy,Roman_Catholic_Church,HIV,0,False,False,0.000642,0.000315,0.003975,0.002173,0.000357,8.0,9.0,9.0,9.0
4,66.000000,True,"[14th_century, Europe, North_America, United_S...",14th_century,John_F._Kennedy,Europe,North_America,United_States,0,False,False,0.000642,0.000315,0.006698,0.002751,0.009774,8.0,8.0,9.0,10.0


In [22]:
embeddings_df = load_embeddings()
games_longer_than_4 = compute_cosine_similarity(games_longer_than_4, embeddings_df, pairs = [['source', 'target'], ['first_click', 'target'], ['second_click', 'target'], ['third_click', 'target']])
games_longer_than_4.head()

Loaded 4604 embeddings in df of shape (4604, 1)


,duration,finished,path,source,target,first_click,second_click,third_click,num_back,second_is_<,...,pagerank_second_click,pagerank_third_click,fame_score_source,fame_score_first_click,fame_score_second_click,fame_score_third_click,cosine_sim_source_target,cosine_sim_first_click_target,cosine_sim_second_click_target,cosine_sim_third_click_target
0,62.250000,True,"[14th_century, 15th_century, 16th_century, Pac...",14th_century,African_slave_trade,15th_century,16th_century,Pacific_Ocean,0,False,...,0.001223,0.001408,8.0,8.0,8.0,9.0,0.202444,0.187263,0.261171,0.015896
1,66.000000,True,"[14th_century, Europe, Africa, Atlantic_slave_...",14th_century,African_slave_trade,Europe,Africa,Atlantic_slave_trade,0,False,...,0.003321,0.000145,8.0,8.0,9.0,6.0,0.202444,0.146602,0.387016,0.485815
2,59.142857,True,"[14th_century, Niger, Nigeria, British_Empire,...",14th_century,African_slave_trade,Niger,Nigeria,British_Empire,0,False,...,0.000630,0.001197,8.0,6.0,8.0,8.0,0.202444,0.309651,0.293009,0.172144
3,87.500000,True,"[14th_century, Italy, Roman_Catholic_Church, H...",14th_century,John_F._Kennedy,Italy,Roman_Catholic_Church,HIV,0,False,...,0.002173,0.000357,8.0,9.0,9.0,9.0,0.079502,0.037840,0.108953,0.012576
4,66.000000,True,"[14th_century, Europe, North_America, United_S...",14th_century,John_F._Kennedy,Europe,North_America,United_States,0,False,...,0.002751,0.009774,8.0,8.0,9.0,10.0,0.079502,-0.128314,-0.044935,0.170961


In [23]:
link_proba = load_link_proba()
link_proba.head()
links_df = load_links_df()
link_proba.isna().sum()
#print(print(games_longer_than_4.loc[3167]))
#links[(links['source'] == 'Finland') & (links['target'] == 'Åland')]

Loaded 119882 links in df of shape (119882, 2)


link_probability    0
dtype: int64

In [ ]:
combs = []
seen_tuples = set() 

for i, row in games_longer_than_4.iterrows():
    pair1 = (row['source'], row['first_click'])
    if pair1 not in link_proba.index and pair1 not in seen_tuples:
        combs.append((i, *pair1))
        seen_tuples.add(pair1)

    pair2 = (row['first_click'], row['second_click'])
    sec_is_b = row['second_is_<']
    if pair2 not in link_proba.index and not sec_is_b and pair2 not in seen_tuples:
        combs.append((i, *pair2))
        seen_tuples.add(pair2)
    
    pair3 = (row['second_click'], row['third_click'])
    third_is_b = row['third_is_<']
    if pair3 not in link_proba.index and not third_is_b and pair3 not in seen_tuples:
        combs.append((i, *pair3))
        seen_tuples.add(pair3)
indexes = []
for i in combs:
    print(i)
    indexes.append(i[0])
print(indexes)

In [25]:
games_longer_than_4.drop(index = indexes, inplace = True)

In [26]:
games_longer_than_4['second_click'] = games_longer_than_4.apply(lambda row: '<' if row['second_is_<'] else row['second_click'], axis = 1)
games_longer_than_4['third_click'] = games_longer_than_4.apply(lambda row: '<' if row['third_is_<'] else row['third_click'], axis = 1)

In [27]:
print((games_longer_than_4['first_click'] == '<').sum())
print((games_longer_than_4['second_click'] == '<').sum())
print((games_longer_than_4['third_click'] == '<').sum())

0
1235
1303


In [28]:
games_longer_than_4 = add_link_proba_info(games_longer_than_4, link_proba, pairs = [['source', 'first_click'], ['first_click', 'second_click'], ['second_click', 'third_click']], names = ['first_link', 'second_link', 'third_link'])
games_longer_than_4.head()

,duration,finished,path,source,target,first_click,second_click,third_click,num_back,second_is_<,...,fame_score_first_click,fame_score_second_click,fame_score_third_click,cosine_sim_source_target,cosine_sim_first_click_target,cosine_sim_second_click_target,cosine_sim_third_click_target,first_link,second_link,third_link
0,62.250000,True,"[14th_century, 15th_century, 16th_century, Pac...",14th_century,African_slave_trade,15th_century,16th_century,Pacific_Ocean,0,False,...,8.0,8.0,9.0,0.202444,0.187263,0.261171,0.015896,0.9,0.60,0.2
1,66.000000,True,"[14th_century, Europe, Africa, Atlantic_slave_...",14th_century,African_slave_trade,Europe,Africa,Atlantic_slave_trade,0,False,...,8.0,9.0,6.0,0.202444,0.146602,0.387016,0.485815,0.9,0.30,0.7
2,59.142857,True,"[14th_century, Niger, Nigeria, British_Empire,...",14th_century,African_slave_trade,Niger,Nigeria,British_Empire,0,False,...,6.0,8.0,8.0,0.202444,0.309651,0.293009,0.172144,0.1,0.90,0.3
3,87.500000,True,"[14th_century, Italy, Roman_Catholic_Church, H...",14th_century,John_F._Kennedy,Italy,Roman_Catholic_Church,HIV,0,False,...,9.0,9.0,9.0,0.079502,0.037840,0.108953,0.012576,0.4,0.25,0.2
4,66.000000,True,"[14th_century, Europe, North_America, United_S...",14th_century,John_F._Kennedy,Europe,North_America,United_States,0,False,...,8.0,9.0,10.0,0.079502,-0.128314,-0.044935,0.170961,0.9,0.10,0.9


In [29]:
print((games_longer_than_4['first_click'] == '<').sum())
print((games_longer_than_4['second_click'] == '<').sum())
print((games_longer_than_4['third_click'] == '<').sum())

0
1235
1303


In [30]:
articles_df = load_article_df()
adj_matrix = construct_adjecency_matrix(links_df, articles_df['article_name'].tolist())
adj_list = from_adjacency_matrix_to_list(adj_matrix)
index_mapping = generate_inverse_index_mapping(adj_list)
pair_data = load_pair_data_with_multiindex('../src/data/pair_stats.txt', index_mapping)
pair_data.head()

Loaded 4604 articles in df of shape (4604, 1)


shortest_path_length  shortest_path_count  \
source       target                                                    
10th_century 10th_century                     0                    1   
             11th_century                     1                    1   
             12th_century                     2                    5   
             13th_century                     2                    4   
             14th_century                     2                    4   

                           max_sp_node_degree  max_sp_avg_node_degree  \
source       target                                                     
10th_century 10th_century                   0                       0   
             11th_century                  26                      13   
             12th_century                  48                      24   
             13th_century                  79                      35   
             14th_century                  53                      26   

                           avg_sp_avg_node_degree  one_longer_path_count  \
source       target                                                        
10th_century 10th_century                       0                      0   
             11th_century                      13                      2   
             12th_century                      23                    121   
             13th_century                      28                    131   
             14th_century                      22                    113   

                           max_ol_node_degree  max_ol_avg_node_degree  \
source       target                                                     
10th_century 10th_century                   0                       0   
             11th_century                  62                      29   
             12th_century                 180                      93   
             13th_century                 169                      84   
             14th_century                 169                      84   

                           avg_ol_avg_node_degree  two_longer_path_count  \
source       target                                                        
10th_century 10th_century                       0                      0   
             11th_century                      26                      7   
             12th_century                      37                     30   
             13th_century                      39                     30   
             14th_century                      37                     30   

                           max_tl_node_degree  max_tl_avg_node_degree  
source       target                                                    
10th_century 10th_century                   0                       0  
             11th_century                 112                      46  
             12th_century                 169                      70  
             13th_century                 169                      71  
             14th_century                 169                      70

In [31]:
pair_data.columns

Index(['shortest_path_length', 'shortest_path_count', 'max_sp_node_degree',
       'max_sp_avg_node_degree', 'avg_sp_avg_node_degree',
       'one_longer_path_count', 'max_ol_node_degree', 'max_ol_avg_node_degree',
       'avg_ol_avg_node_degree', 'two_longer_path_count', 'max_tl_node_degree',
       'max_tl_avg_node_degree'],
      dtype='object')

In [32]:
games_longer_than_4 = add_pair_data(games_longer_than_4, pair_data, pairs =[['source', 'target'], ['first_click', 'target'], ['second_click', 'target'], ['third_click', 'target']], names = ["source", "first", "second", "third"], data = ['shortest_path_length', 'shortest_path_count']) 

Dropped 17 games without link statistics


In [33]:
games_longer_than_4.head()

,duration,finished,path,source,target,first_click,second_click,third_click,num_back,second_is_<,...,second_link,third_link,shortest_path_length_source,shortest_path_count_source,shortest_path_length_first,shortest_path_count_first,shortest_path_length_second,shortest_path_count_second,shortest_path_length_third,shortest_path_count_third
0,62.250000,True,"[14th_century, 15th_century, 16th_century, Pac...",14th_century,African_slave_trade,15th_century,16th_century,Pacific_Ocean,0,False,...,0.60,0.2,3.0,3.0,3.0,3.0,3.0,6.0,3.0,3.0
1,66.000000,True,"[14th_century, Europe, Africa, Atlantic_slave_...",14th_century,African_slave_trade,Europe,Africa,Atlantic_slave_trade,0,False,...,0.30,0.7,3.0,3.0,3.0,7.0,2.0,1.0,1.0,1.0
2,59.142857,True,"[14th_century, Niger, Nigeria, British_Empire,...",14th_century,African_slave_trade,Niger,Nigeria,British_Empire,0,False,...,0.90,0.3,3.0,3.0,3.0,4.0,2.0,1.0,3.0,7.0
3,87.500000,True,"[14th_century, Italy, Roman_Catholic_Church, H...",14th_century,John_F._Kennedy,Italy,Roman_Catholic_Church,HIV,0,False,...,0.25,0.2,3.0,18.0,2.0,1.0,2.0,1.0,2.0,2.0
4,66.000000,True,"[14th_century, Europe, North_America, United_S...",14th_century,John_F._Kennedy,Europe,North_America,United_States,0,False,...,0.10,0.9,3.0,18.0,2.0,2.0,2.0,2.0,1.0,1.0


In [34]:
games_longer_than_4.columns

Index(['duration', 'finished', 'path', 'source', 'target', 'first_click',
       'second_click', 'third_click', 'num_back', 'second_is_<', 'third_is_<',
       'pagerank_source', 'pagerank_target', 'pagerank_first_click',
       'pagerank_second_click', 'pagerank_third_click', 'fame_score_source',
       'fame_score_first_click', 'fame_score_second_click',
       'fame_score_third_click', 'cosine_sim_source_target',
       'cosine_sim_first_click_target', 'cosine_sim_second_click_target',
       'cosine_sim_third_click_target', 'first_link', 'second_link',
       'third_link', 'shortest_path_length_source',
       'shortest_path_count_source', 'shortest_path_length_first',
       'shortest_path_count_first', 'shortest_path_length_second',
       'shortest_path_count_second', 'shortest_path_length_third',
       'shortest_path_count_third'],
      dtype='object')

In [35]:
games_longer_than_4.drop(columns = ['path', 'source', 'target', 'first_click',
       'second_click', 'third_click', 'num_back'], inplace = True)

In [36]:
games_longer_than_4.head()

,duration,finished,second_is_<,third_is_<,pagerank_source,pagerank_target,pagerank_first_click,pagerank_second_click,pagerank_third_click,fame_score_source,...,second_link,third_link,shortest_path_length_source,shortest_path_count_source,shortest_path_length_first,shortest_path_count_first,shortest_path_length_second,shortest_path_count_second,shortest_path_length_third,shortest_path_count_third
0,62.250000,True,False,False,0.000642,0.000030,0.001024,0.001223,0.001408,8.0,...,0.60,0.2,3.0,3.0,3.0,3.0,3.0,6.0,3.0,3.0
1,66.000000,True,False,False,0.000642,0.000030,0.006698,0.003321,0.000145,8.0,...,0.30,0.7,3.0,3.0,3.0,7.0,2.0,1.0,1.0,1.0
2,59.142857,True,False,False,0.000642,0.000030,0.000408,0.000630,0.001197,8.0,...,0.90,0.3,3.0,3.0,3.0,4.0,2.0,1.0,3.0,7.0
3,87.500000,True,False,False,0.000642,0.000315,0.003975,0.002173,0.000357,8.0,...,0.25,0.2,3.0,18.0,2.0,1.0,2.0,1.0,2.0,2.0
4,66.000000,True,False,False,0.000642,0.000315,0.006698,0.002751,0.009774,8.0,...,0.10,0.9,3.0,18.0,2.0,2.0,2.0,2.0,1.0,1.0


In [37]:
games_longer_than_4.columns

Index(['duration', 'finished', 'second_is_<', 'third_is_<', 'pagerank_source',
       'pagerank_target', 'pagerank_first_click', 'pagerank_second_click',
       'pagerank_third_click', 'fame_score_source', 'fame_score_first_click',
       'fame_score_second_click', 'fame_score_third_click',
       'cosine_sim_source_target', 'cosine_sim_first_click_target',
       'cosine_sim_second_click_target', 'cosine_sim_third_click_target',
       'first_link', 'second_link', 'third_link',
       'shortest_path_length_source', 'shortest_path_count_source',
       'shortest_path_length_first', 'shortest_path_count_first',
       'shortest_path_length_second', 'shortest_path_count_second',
       'shortest_path_length_third', 'shortest_path_count_third'],
      dtype='object')

In [38]:
features_1 = ['duration', 'second_is_<', 'third_is_<', 'pagerank_source',
       'pagerank_target', 'pagerank_first_click', 'pagerank_second_click',
       'pagerank_third_click', 'fame_score_source', 'fame_score_first_click',
       'fame_score_second_click', 'fame_score_third_click', 'first_link',
       'second_link', 'third_link', 'shortest_path_length_source',
       'shortest_path_count_source', 'shortest_path_length_first',
       'shortest_path_count_first', 'shortest_path_length_second',
       'shortest_path_count_second', 'shortest_path_length_third',
       'shortest_path_count_third',  'cosine_sim_source_target',
       'cosine_sim_first_click_target', 'cosine_sim_second_click_target',
       'cosine_sim_third_click_target']

In [39]:
model_1 = LogisticRegression(games_longer_than_4, features_1)
model_1.fit()

Class distribution: finished
False    0.5
True     0.5
Name: proportion, dtype: float64
Total number of samples: 13694


KeyError: 'n'

In [40]:
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               finished   No. Observations:                10955
Model:                          Logit   Df Residuals:                    10928
Method:                           MLE   Df Model:                           26
Date:                Fri, 13 Dec 2024   Pseudo R-squ.:                  0.2111
Time:                        09:43:55   Log-Likelihood:                -5990.5
converged:                       True   LL-Null:                       -7593.4
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.1963      0.060     -3.264      0.001      -0.314      -0.078
x2             0.0114      0.041      0.279      0.780      -0.068       0.091
x3            -0.8364      0.034    -24.347      0.000      -0.904      -0.769
x4             0.0046      0.024      0.192      0.848      -0.043       0.052
x5             0.3139      0.038      8.183      0.000       0.239       0.389
x6            -0.0016      0.026     -0.064      0.949      -0.052       0.049
x7            -0.0417      0.026     -1.629      0.103      -0.092       0.008
x8            -0.1189      0.025     -4.790      0.000      -0.168      -0.070
x9             0.0443      0.024      1.869      0.062      -0.002       0.091
x10           -0.0057      0.024     -0.235      0.814      -0.054       0.042
x11            0.0022      0.025      0.090      0.928      -0.046       0.050
x12           -0.0269      0.024     -1.098      0.272      -0.075       0.021
x13            0.0204      0.023      0.899      0.368      -0.024       0.065
x14           -0.0093      0.024     -0.384      0.701      -0.057       0.038
x15           -0.0036      0.026     -0.139      0.890      -0.055       0.048
x16            0.0642      0.039      1.651      0.099      -0.012       0.140
x17           -0.0281      0.027     -1.059      0.290      -0.080       0.024
x18           -0.0043      0.041     -0.105      0.916      -0.085       0.076
x19            0.0667      0.026      2.523      0.012       0.015       0.119
x20           -0.1028      0.052     -1.979      0.048      -0.205      -0.001
x21            0.0716      0.026      2.732      0.006       0.020       0.123
x22           -1.4319      0.048    -29.688      0.000      -1.526      -1.337
x23            0.1902      0.025      7.476      0.000       0.140       0.240
x24           -0.0440      0.025     -1.756      0.079      -0.093       0.005
x25           -0.0050      0.028     -0.181      0.857      -0.060       0.050
x26           -0.0622      0.031     -2.025      0.043      -0.122      -0.002
x27            0.1026      0.032      3.257      0.001       0.041       0.164
==============================================================================
"""

In [41]:
features_2 = ['duration', 'second_is_<','third_is_<',
       'pagerank_target',  'pagerank_second_click',
       'pagerank_third_click', 'shortest_path_length_source', 
       'shortest_path_count_first', 'shortest_path_length_second',
       'shortest_path_count_second', 'shortest_path_length_third',
       'shortest_path_count_third',  'cosine_sim_source_target',
       'cosine_sim_second_click_target',
       'cosine_sim_third_click_target']

In [42]:
model_2 = LogisticRegression(games_longer_than_4, features_2)
model_2.fit()

Class distribution: finished
False    0.5
True     0.5
Name: proportion, dtype: float64
Total number of samples: 13694
Optimization terminated successfully.
         Current function value: 0.547156
         Iterations 7
Training Set Metrics:
Threshold:   0.5750
F1 Score:    0.7163
Precision:   0.7206
Accuracy:    0.7173
              precision    recall  f1-score   support

       False     0.7095    0.7985    0.7514      1370
        True     0.7694    0.6728    0.7178      1369

    accuracy                         0.7357      2739
   macro avg     0.7394    0.7356    0.7346      2739
weighted avg     0.7394    0.7357    0.7346      2739



In [43]:
model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               finished   No. Observations:                10955
Model:                          Logit   Df Residuals:                    10940
Method:                           MLE   Df Model:                           14
Date:                Fri, 13 Dec 2024   Pseudo R-squ.:                  0.2106
Time:                        09:43:57   Log-Likelihood:                -5994.1
converged:                       True   LL-Null:                       -7593.4
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.1983      0.060     -3.282      0.001      -0.317      -0.080
x2             0.0208      0.037      0.555      0.579      -0.053       0.094
x3            -0.8323      0.032    -25.619      0.000      -0.896      -0.769
x4             0.3081      0.037      8.387      0.000       0.236       0.380
x5            -0.0420      0.024     -1.729      0.084      -0.090       0.006
x6            -0.1291      0.023     -5.584      0.000      -0.174      -0.084
x7             0.0368      0.030      1.227      0.220      -0.022       0.096
x8             0.0639      0.022      2.864      0.004       0.020       0.108
x9            -0.0940      0.048     -1.941      0.052      -0.189       0.001
x10            0.0700      0.026      2.721      0.006       0.020       0.120
x11           -1.4251      0.048    -29.847      0.000      -1.519      -1.332
x12            0.1888      0.025      7.454      0.000       0.139       0.238
x13           -0.0458      0.024     -1.935      0.053      -0.092       0.001
x14           -0.0629      0.029     -2.142      0.032      -0.121      -0.005
x15            0.1038      0.031      3.369      0.001       0.043       0.164
==============================================================================
"""

Let's see if using an average of node values helps.

pagerank target, duration, shortest path length zadnji i shortest path count zadnji, cosine similarity zadnji, max pagerank dosad, brback/brukup,  

In [72]:
games_longer_than_4_bis = games_longer_than_4.copy()
games_longer_than_4_bis['max_pgrank'] = games_longer_than_4_bis.apply(lambda row: max(row['pagerank_source'], row['pagerank_first_click'], row['pagerank_second_click'], row['pagerank_third_click']), axis = 1)
games_longer_than_4_bis['back'] = games_longer_than_4_bis.apply(lambda row: (row['second_is_<'] + row['third_is_<'])/2, axis = 1)
games_longer_than_4_bis['cos_diff'] = games_longer_than_4_bis.apply(lambda row: (-row['cosine_sim_source_target'] + row['cosine_sim_third_click_target']), axis = 1)
games_longer_than_4_bis.head()

,duration,finished,second_is_<,third_is_<,pagerank_source,pagerank_target,pagerank_first_click,pagerank_second_click,pagerank_third_click,fame_score_source,...,shortest_path_count_source,shortest_path_length_first,shortest_path_count_first,shortest_path_length_second,shortest_path_count_second,shortest_path_length_third,shortest_path_count_third,max_pgrank,back,cos_diff
0,62.250000,True,False,False,0.000642,0.000030,0.001024,0.001223,0.001408,8.0,...,3.0,3.0,3.0,3.0,6.0,3.0,3.0,0.001408,0.0,-0.186548
1,66.000000,True,False,False,0.000642,0.000030,0.006698,0.003321,0.000145,8.0,...,3.0,3.0,7.0,2.0,1.0,1.0,1.0,0.006698,0.0,0.283370
2,59.142857,True,False,False,0.000642,0.000030,0.000408,0.000630,0.001197,8.0,...,3.0,3.0,4.0,2.0,1.0,3.0,7.0,0.001197,0.0,-0.030300
3,87.500000,True,False,False,0.000642,0.000315,0.003975,0.002173,0.000357,8.0,...,18.0,2.0,1.0,2.0,1.0,2.0,2.0,0.003975,0.0,-0.066926
4,66.000000,True,False,False,0.000642,0.000315,0.006698,0.002751,0.009774,8.0,...,18.0,2.0,2.0,2.0,2.0,1.0,1.0,0.009774,0.0,0.091459


In [73]:
games_longer_than_4_bis.columns

Index(['duration', 'finished', 'second_is_<', 'third_is_<', 'pagerank_source',
       'pagerank_target', 'pagerank_first_click', 'pagerank_second_click',
       'pagerank_third_click', 'fame_score_source', 'fame_score_first_click',
       'fame_score_second_click', 'fame_score_third_click',
       'cosine_sim_source_target', 'cosine_sim_first_click_target',
       'cosine_sim_second_click_target', 'cosine_sim_third_click_target',
       'first_link', 'second_link', 'third_link',
       'shortest_path_length_source', 'shortest_path_count_source',
       'shortest_path_length_first', 'shortest_path_count_first',
       'shortest_path_length_second', 'shortest_path_count_second',
       'shortest_path_length_third', 'shortest_path_count_third', 'max_pgrank',
       'back', 'cos_diff'],
      dtype='object')

In [74]:
games_longer_than_4_bis.drop(columns = ['second_is_<', 'third_is_<', 'pagerank_source',
      'pagerank_first_click', 'pagerank_second_click',
        'fame_score_source', 'fame_score_first_click',
       'fame_score_second_click', 'fame_score_third_click',
       'cosine_sim_source_target', 'cosine_sim_first_click_target',
       'cosine_sim_second_click_target', 'pagerank_third_click',
       'first_link', 'second_link', 'third_link',
       'shortest_path_length_source', 'shortest_path_count_source',
       'shortest_path_length_first', 'shortest_path_count_first',
       'shortest_path_length_second', 'shortest_path_count_second',
      ], inplace=True)

In [75]:
games_longer_than_4_bis.head()

,duration,finished,pagerank_target,cosine_sim_third_click_target,shortest_path_length_third,shortest_path_count_third,max_pgrank,back,cos_diff
0,62.250000,True,0.000030,0.015896,3.0,3.0,0.001408,0.0,-0.186548
1,66.000000,True,0.000030,0.485815,1.0,1.0,0.006698,0.0,0.283370
2,59.142857,True,0.000030,0.172144,3.0,7.0,0.001197,0.0,-0.030300
3,87.500000,True,0.000315,0.012576,2.0,2.0,0.003975,0.0,-0.066926
4,66.000000,True,0.000315,0.170961,1.0,1.0,0.009774,0.0,0.091459


In [76]:
games_longer_than_4_bis.columns

Index(['duration', 'finished', 'pagerank_target',
       'cosine_sim_third_click_target', 'shortest_path_length_third',
       'shortest_path_count_third', 'max_pgrank', 'back', 'cos_diff'],
      dtype='object')

In [77]:
features_3 = ['duration', 'pagerank_target',
       'cosine_sim_third_click_target', 'shortest_path_length_third',
       'shortest_path_count_third', 'max_pgrank','back', 'cos_diff']

In [78]:
model_3 = LogisticRegression(games_longer_than_4_bis, features_3)
model_3.fit()

Class distribution: finished
False    0.5
True     0.5
Name: proportion, dtype: float64
Total number of samples: 13694
Optimization terminated successfully.
         Current function value: 0.576916
         Iterations 7
Training Set Metrics:
Threshold:   0.5400
F1 Score:    0.7062
Precision:   0.7086
Accuracy:    0.7068
              precision    recall  f1-score   support

       False     0.6922    0.7599    0.7244      1370
        True     0.7336    0.6618    0.6959      1369

    accuracy                         0.7108      2739
   macro avg     0.7129    0.7108    0.7101      2739
weighted avg     0.7129    0.7108    0.7101      2739



In [79]:
model_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               finished   No. Observations:                10955
Model:                          Logit   Df Residuals:                    10947
Method:                           MLE   Df Model:                            7
Date:                Fri, 13 Dec 2024   Pseudo R-squ.:                  0.1677
Time:                        10:20:07   Log-Likelihood:                -6320.1
converged:                       True   LL-Null:                       -7593.4
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.2446      0.065     -3.790      0.000      -0.371      -0.118
x2             0.5405      0.035     15.308      0.000       0.471       0.610
x3             0.1311      0.050      2.615      0.009       0.033       0.229
x4            -0.8788      0.032    -27.423      0.000      -0.942      -0.816
x5             0.1091      0.024      4.538      0.000       0.062       0.156
x6            -0.0423      0.022     -1.930      0.054      -0.085       0.001
x7            -0.4291      0.028    -15.519      0.000      -0.483      -0.375
x8             0.1027      0.049      2.091      0.036       0.006       0.199
==============================================================================
"""